<a href="https://colab.research.google.com/github/fneaplle/ML/blob/main/BatchNormalization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import tensorflow as tf
import numpy as np

In [15]:
class BatchBlock(tf.keras.layers.Layer):
  def __init__(self):
    super().__init__()

  def build(self, input_shape):
    dim=input_shape[-1]
    self.gamma=self.add_weight(shape=(dim,), initializer='ones', trainable=True)
    self.beta=self.add_weight(shape=(dim,), initializer='zeros', trainable=True)
    self.var=self.add_weight(shape=(dim,), initializer='ones', trainable=False)
    self.mean=self.add_weight(shape=(dim,), initializer='zeros', trainable=False)

  def call(self, inp, training=False):
    if training:
      mean, var = tf.nn.moments(inp, axes=[i for i in range(
          inp.shape.rank - 1)])
      normalized = (inp - mean) / var
      self.var.assign(self.var * 0.9 + var*0.1)
      self.mean.assign(self.mean*0.9 + mean*0.1)
    else:
      normalized = (inp-self.mean)/self.var
    return normalized*self.gamma+self.beta

In [16]:
class mlp(tf.keras.Model):
  def __init__(self):
    super().__init__()

    self.bb=BatchBlock()
    self.db1=tf.keras.layers.Dense(32)
    self.fb=tf.keras.layers.Dense(1)

  def call(self, inp):
    x=self.bb(inp)
    x=self.db1(inp)
    result=self.fb(x)

    return result

model=mlp()

In [ ]:
X=tf.random.normal((32,10,5))
model(X)

In [28]:
model.compile(optimizer='rmsprop',
              loss=tf.keras.losses.mean_squared_error)

In [29]:
X=tf.random.normal((32, 10, 5))
y=tf.random.normal((32,10, 1))

In [ ]:
model.trainable_variables

In [30]:
model.fit(X,y,batch_size=30,epochs=10)

Epoch 1/10
2/2 [==============================] - 1s 3ms/step - loss: 1.4060
Epoch 2/10
2/2 [==============================] - 0s 4ms/step - loss: 1.2780
Epoch 3/10
2/2 [==============================] - 0s 3ms/step - loss: 1.2207
Epoch 4/10
2/2 [==============================] - 0s 4ms/step - loss: 1.1575
Epoch 5/10
2/2 [==============================] - 0s 4ms/step - loss: 1.1350
Epoch 6/10
2/2 [==============================] - 0s 4ms/step - loss: 1.1132
Epoch 7/10
2/2 [==============================] - 0s 4ms/step - loss: 1.0911
Epoch 8/10
2/2 [==============================] - 0s 4ms/step - loss: 1.0653
Epoch 9/10
2/2 [==============================] - 0s 6ms/step - loss: 1.0290
Epoch 10/10
2/2 [==============================] - 0s 3ms/step - loss: 1.0298
